# Telangana RERA Scraper

1. Open the website ™ (and import stuff)

In [30]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import platform

browser: WebDriver

if platform == 'Darwin':
	browser = webdriver.Safari()
else:
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")

browser.get("http://rerait.telangana.gov.in/SearchList/Search")
page_size = browser.find_element_by_id('PageSize')

# Uncomment this when not in development; it'll use 100 elements per page instead of the default 10
# page_size.send_keys("0")

counter: int = 1

2. Scrape the 100 entries on the page

In [32]:
from selenium.webdriver.remote.webelement import WebElement

row_xpath = '//*[@id="gridview"]/div[1]/div/table/tbody/tr'

listOfVars = []

rows: list[WebElement] = browser.find_elements_by_xpath(row_xpath)
for row_i, row in enumerate(rows):
	cells: list[WebElement] = row.find_elements_by_xpath('{}[{}]/td'.format(row_xpath, row_i + 1))
	# Intentionally using camelCase instead of snake_case as the schema uses camelCase
	projectName = cells[1].get_attribute('innerHTML')
	promoterName = cells[2].get_attribute('innerHTML')
	lastModifiedDate = cells[3].get_attribute('innerHTML')
	dictionary = {"projectName": projectName, "promoterName": promoterName, "lastModifiedDate": lastModifiedDate}
	listOfVars.append(dictionary)

file = open("data.json", "w")
file.write(json.dumps(listOfVars, indent=2))

file.close()

counter += 1 # Use this in the file name

TypeError: write() argument must be str, not int